In [ ]:
!pip install gurobipy

In [ ]:
import gurobipy as gp
from gurobipy import GRB

In [ ]:
# Declara e inicializa el modelo
m = gp.Model('Modelo - Múltiple Soluciones')

In [ ]:
# Crea las variables de decisión para el modelo Modelo1
x1 = m.addVar(name='x1')
x2 = m.addVar(name='x2')
s1 = m.addVar(name='s1')
s2 = m.addVar(name='s2')
# Integra las variables
m.update()

In [ ]:
m.addConstr(-2*x1 + 5*x2 + s1 == 20, "r0")
m.addConstr(3*x1 + 4*x2 + s2  == 24, "r1")
m.update()

In [ ]:
m.setObjective(6*x1 + 8*x2, GRB.MAXIMIZE)
m.update()

In [ ]:
m.optimize()

In [ ]:
## Revisa la condición de optimalidad
if m.status == GRB.OPTIMAL:
    print('Optimal objective: %g' % m.objVal)

In [ ]:
## Valor de la variables en el óptimo
for v in m.getVars():
    print(v.varName, ' = ', '{0:.2f}'.format(v.x))

In [ ]:
## Valor de los costos reducidos en el óptimo
for rc1 in m.getVars():
    print('Costo reducido de ', rc1.VarName,' = ', '{0:.2f}'.format(rc1.getAttr(GRB.Attr.RC)))

Hemos identificado que la solución óptima tiene infinitas soluciones, esto debido a que en existe una variable no-basica ($x_2$) que tiene un costo reducido igual a CERO. Por lo tanto, esta variable $x_2$ puede entrar a la base y no alterar la función objetivo, es decir, otro vértice óptimo, y con ello infinitas soluciones.

**Una forma de PIVOTEO**

Aun cuando GUROBI no ofrece una posibilidad directa de pivotear cunado existen infinitas soluciones, una forma de hacerlo es considerar:

*   Identificar la variable no-básica que tiene costo reducido cero, en este caso es $x_2$, para ello intentaremos sacar la variable $s_1$ de la base y ver si $x_2$ entra a ella
*   Agregar una restricción adicional en la cual se setea $s_1$ a ceroayor** que CERO
* Actualizar el modelo y optimizar nuevamente

Así, hay que revisar que el valor de la función objetivo no cambie (infinitas soluciones) y así entregará un nuevo punto extremo o vértice



In [ ]:
m.addConstr(s1 == 0, name='r*')
m.update()
m.display()

In [ ]:
m.optimize()

In [ ]:
## Valor de la variables en el óptimo
for v in m.getVars():
    print(v.varName, ' = ', '{0:.2f}'.format(v.x))

**Múltiples soluciones**

Entonces, tenemos las siguientes soluciones con el mismo valor de la función objetivo

*   Solución 1: $\vec{x}_1 = (x_1, x_2, s_1, s_2) = (8, 0, 36, 0)$
*   Solución 2: $\vec{x}_2 = (x_1, x_2, s_1, s_2) = (1.74, 4.70, 0, 0)$

Luego, las infinitas soluciones corresponde:

   $\vec{x} = \lambda \: \vec{x}_1 + (1 - λ) \: \vec{x}_2, \: con \: 0 \le λ \le 1$


In [ ]:
## Valor de PI en el óptimo
for res1 in m.getConstrs():
    print('Valor PI en ', res1.ConstrName,' = ', '{0:.2f}'.format(res1.getAttr(GRB.Attr.Pi)))